In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[1]") \
    .appName("SparkByExamples.com") \
    .getOrCreate()

22/04/10 17:48:09 WARN Utils: Your hostname, DSGPU05 resolves to a loopback address: 127.0.1.1; using 10.10.11.64 instead (on interface eno1)
22/04/10 17:48:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/10 17:48:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/04/10 17:48:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
classics = pd.read_csv("data/classics_raw.csv")

NameError: name 'pd' is not defined

In [ ]:
classics.head()

In [ ]:
import string

In [5]:
lines = (
    classics.text.str.split("\n")
    .apply(pd.Series)
    .melt(ignore_index=False, value_name="line", var_name="text_unit_id")
    .dropna()
)

NameError: name 'classics' is not defined

In [6]:
del classics["text"]
print(classics.merge(lines, left_index=True, right_index=True))

NameError: name 'classics' is not defined

In [ ]:
classics = classics.merge(lines, left_index=True, right_index=True).dropna()

In [ ]:
classics_df = spark.createDataFrame(classics)
classics_df.printSchema()
classics_df.show()

In [ ]:
from pyspark.sql.functions import udf, col, StringType, column
import re
def split(value):
    punc = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    for ch in punc:
        value = value.replace(ch, ' ')
    return value.lower()
def removeDigit(value):
    digits = re.compile("\{\d+:\d+\}")
    return digits.sub("", value)
splitPunct = udf(lambda z: removeDigit(split(z)), StringType())
removeDigits = udf(lambda z: removeDigit(z), StringType())

In [ ]:
classics_df = classics_df.select(("*"), \
    splitPunct(col("line").alias("cleaned")))

In [ ]:
classics_df.show()

In [ ]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover
re_tokenizer = RegexTokenizer()
re_tokenizer.setInputCol("<lambda>(line AS cleaned)").setOutputCol("tokens")

In [ ]:
classics_df = re_tokenizer.transform(classics_df)
classics_df.show()

In [ ]:
from pyspark.ml.linalg import Vectors, SparseVector
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer
lda = LDA()
lda.setMaxIter(10)
cv = CountVectorizer()

In [ ]:
cv.setInputCol("tokens")
cv.setOutputCol("vectors")
model = cv.fit(classics_df)

In [ ]:
model.transform(classics_df).show()

In [ ]:
sorted(model.vocabulary)[:100]

In [ ]:
td= classics_df.select(col("tokens"))
from pyspark.ml.feature import CountVectorizer
vectorizer = CountVectorizer(inputCol="tokens", outputCol="bag_of_words")
vectorizer_transformer = vectorizer.fit(td)

In [ ]:
?vectorizer_transformer

In [ ]:
my_vector = vectorizer_transformer.transform(classics_df)

In [ ]:
my_vector.select("bag_of_words").show()

In [ ]:
bag_words = my_vector.select("bag_of_words")
lda.fit(bag_words)